# **Imports and Downloads**

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --upgrade --force-reinstall --quiet
!pip install --upgrade pennylane pennylane-lightning

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [ ]:
# Core libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

import numpy as np
import pandas as pd
import pickle
import math
import re
import os
import random
import time
import matplotlib.pyplot as plt
from collections import Counter

# Quantum computing
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import AdamOptimizer

# NLP and preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams

# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
pnp.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# **Data Loading and Preprocessing**

In [ ]:
with open("/content/drive/MyDrive/QML-Research/Data/sentiment labelled sentences/amazon_cells_labelled.txt", "r") as f:
    lines = f.readlines()

sentences = [line.split("\t")[0] for line in lines]
labels = [int(line.split("\t")[1]) for line in lines]

In [ ]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
domain_neutral_words = {
    "phone", "product", "battery", "headset", "quality", "one", "use"
}
stop_words.update(domain_neutral_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean_and_tokenize(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

cleaned_sents = [clean_and_tokenize(sentence) for sentence in sentences]

In [ ]:
max_len = 10
for i in range(len(cleaned_sents)):
  if (len(cleaned_sents[i]) < max_len):
    cleaned_sents[i] += ["<PAD>"] * (max_len - len(cleaned_sents[i]))
  else:
    cleaned_sents[i] = cleaned_sents[i][:max_len]

# **GloVE Word Embeddings**

In [ ]:
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

In [ ]:
glove_path = '/content/drive/MyDrive/QML-Research/Data/glove.6B.100d.txt'
glove = load_glove_embeddings(glove_path)

# **AutoEncoder**

In [ ]:
class GloVeAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(GloVeAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
all_words = list(glove.keys())
all_vectors = np.array([glove[word] for word in all_words])
all_vectors = normalize(all_vectors)
word_tensor = torch.tensor(all_vectors).float()

In [ ]:
latent_dim = 8
epochs = 100
save_path = '/content/drive/MyDrive/QML-Research/Autoencoder/glove_autoencoder_normalized_8.pth'
# save_path = '/content/drive/MyDrive/QML-Research/Autoencoder/glove_autoencoder_normalized_32.pth'

In [ ]:
if os.path.exists(save_path):
    print(f"Loading Autoencoder from {save_path}")
    autoencoder = GloVeAutoencoder(input_dim=100, latent_dim=latent_dim)
    autoencoder.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
else:
    print("Training Autoencoder")
    autoencoder = GloVeAutoencoder(input_dim=100, latent_dim=latent_dim)
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        reconstructed = autoencoder(word_tensor)
        loss = criterion(reconstructed, word_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    torch.save(autoencoder.state_dict(), save_path)
    print(f"Saved Autoencoder serialized model in drive @ {save_path}")

Loading Autoencoder from /content/drive/MyDrive/QML-Research/Autoencoder/glove_autoencoder_normalized_8.pth


In [ ]:
autoencoder.eval()
with torch.no_grad():
    compressed_vectors = autoencoder.encoder(word_tensor).numpy()

reduced_embeddings = {
    word: compressed_vectors[i]
    for i, word in enumerate(all_words)
}

# **Embedding**

In [ ]:
def sentence_to_vec(sentence, embeddings, dim):
    vectors = []
    for word in sentence:
        if word in embeddings:
            vectors.append(embeddings[word])
        else:
            vectors.append(np.zeros(dim))
    return vectors

def embed_sentences(cleaned_sents, embeddings, dim):
    return np.array([sentence_to_vec(tokens, embeddings, dim) for tokens in cleaned_sents])

In [ ]:
X_embed_np = embed_sentences(cleaned_sents, reduced_embeddings, dim=8)
X_embed_np = (X_embed_np - X_embed_np.min()) * (np.pi / (X_embed_np.max() - X_embed_np.min()))

X_embed = torch.tensor(X_embed_np).long()
y_embed = torch.tensor(labels).long()

# **Dataset and DataLoader**

In [ ]:
class AmazonDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_embed, y_embed, test_size=0.2, stratify=y_embed.numpy(), random_state=42
)

train_dataset = AmazonDataset(X_train, y_train)
test_dataset = AmazonDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1)

# **Utility Functions**

In [ ]:
def softmax(logits):
    logits = qml.numpy.array(logits)
    logits = logits - qml.numpy.max(logits)
    exps = qml.numpy.exp(logits)
    return exps / qml.numpy.sum(exps)

def cross_entropy(label, probs):
    p = qml.numpy.clip(probs[label], 1e-10, 1.0)
    return -qml.numpy.log(p)

# def reshape_params(flat_params):
#     wpw = flat_params[:80].reshape((10, 8))
#     clf = flat_params[80:].reshape((8,))
#     return wpw, clf

# **QRNN Model Layers**

In [ ]:
"""
First Variation of QRNN that showed learning during training, Achieving a training accuracy of 67% (Highest)
and a peak testing accuracy of 55.5% with a training duration of 50-60 epochs. The layers have been split
into Unitary and entanglement and have been implemnted as functions. This model processes the whole sequence
(All Timesteps) at once in contrast to the previous iterations that processes one time-step/word at a time.
"""

# def dense_angle_embedding(x, wires):
#     # For 32 dim:
#     # chunk_size = 8
#     chunk_size = 2

#     for i, wire in enumerate(wires):
#         for j in range(chunk_size):
#             idx = i * chunk_size + j
#             qml.RX(x[idx], wires=wire)
#             qml.RY(x[idx], wires=wire)
#             qml.RZ(x[idx], wires=wire)

# def variational_block(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.RY(weights[i], wires=wire)
#         qml.RZ(weights[i + len(wires)], wires=wire)

# def entangle(wires):
#     for i in range(len(wires) - 1):
#         qml.CNOT(wires=[wires[i], wires[i + 1]])
#     qml.CNOT(wires=[wires[-1], wires[0]])

'\nFirst Variation of QRNN that showed learning during training, Achieving a training accuracy of 67% (Highest)\nand a peak testing accuracy of 55.5% with a training duration of 50-60 epochs. The layers have been split\ninto Unitary and entanglement and have been implemnted as functions. This model processes the whole sequence\n(All Timesteps) at once in contrast to the previous iterations that processes one time-step/word at a time.\n'

In [ ]:
"""
The second iteration of the previous model introducing major architectural changes (All layers are implemented as functions):
a. Stacked QRNN (2 layered QRNN), with each QRB having seperate weights (No weight sharing)
b. Interaction layers implemented seperate from the QRBs (Quantum Recurrent Block)
c. Introduced an additional interaction layer (classification layer) before quantum measurement
"""

def input_layer(x_sequence, wires):
    for word_vec in x_sequence:
        for i, wire in enumerate(wires):
            qml.RX(word_vec[2 * i], wires=wire)
            qml.RY(word_vec[2 * i + 1], wires=wire)
            qml.RZ(word_vec[2 * i], wires=wire)

# def qrnn_layer(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.RY(weights[i], wires=wire)
#         qml.RZ(weights[i + len(wires)], wires=wire)

def qrnn_layer(weights, wires):
    for i, wire in enumerate(wires):
        qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=wire)

def interaction_layer(wires):
    for i in range(len(wires)):
        qml.CNOT(wires=[wires[i], wires[(i + 1) % len(wires)]])

# def final_interaction_layer(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.RY(weights[i], wires=wire)
#         qml.RZ(weights[i + len(wires)], wires=wire)

# def final_interaction_layer(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.Rot(*weights[i], wires=wire)

def final_interaction_layer(w, wires):
    for i in range(len(wires) - 1):
        qml.CNOT(wires=[wires[i], wires[i + 1]])

    for i in range(len(wires)):
        qml.Rot(w[i, 0], w[i, 1], w[i, 2], wires=wires[i])

# **Model**

In [ ]:
# def forward(input_sequence, weights_in, weights_h, hidden_init=None):
#     """
#     input_sequence: (10, 32) numpy array
#     weights_in: (6,) numpy array
#     weights_h: (2,) numpy array
#     hidden_init: optional scalar float, default is small random value
#     """
#     if hidden_init is None:
#         hidden = np.random.uniform(-0.1, 0.1)
#     else:
#         hidden = hidden_init

#     for word_vec in input_sequence:
#         logits = quantum_step(word_vec, hidden, weights_in, weights_h)
#         hidden = np.tanh(logits[0] + logits[1])

#     return logits

In [ ]:
def qrnn_model(x_sequence, flat_params, wires):
    w_qrb1 = flat_params[:12].reshape((4, 3))
    w_qrb2 = flat_params[12:24].reshape((4, 3))
    w_qrb3 = flat_params[24:36].reshape((4, 3))
    w_classifier = flat_params[36:].reshape((4, 3))

    input_layer(x_sequence, wires)

    qrnn_layer(w_qrb1, wires)
    # interaction_layer(wires)

    qrnn_layer(w_qrb2, wires)
    # interaction_layer(wires)

    qrnn_layer(w_qrb3, wires)
    # interaction_layer(wires)

    final_interaction_layer(w_classifier, wires)

In [ ]:
# def qrnn_model(x_sequence, flat_params, wires, mode="full"):
#   # Phase 1
#     if mode == "phase1":
#         w_qrb1 = flat_params[:12].reshape((4, 3))
#         w_qrb1 = qml.numpy.array(w_qrb1, requires_grad=True)

#         input_layer(x_sequence, wires)
#         qrnn_layer(w_qrb1, wires)
#         interaction_layer(wires)

#     # Phase 2
#     elif mode == "phase2":
#         w_qrb1_raw = flat_params[:12].reshape((4, 3))
#         w_qrb2_raw = flat_params[12:24].reshape((4, 3))

#         w_qrb1 = qml.numpy.array(w_qrb1_raw, requires_grad=False)
#         w_qrb2 = qml.numpy.array(w_qrb2_raw, requires_grad=True)

#         input_layer(x_sequence, wires)
#         qrnn_layer(w_qrb1, wires)
#         interaction_layer(wires)
#         qrnn_layer(w_qrb2, wires)
#         interaction_layer(wires)


#     # Phase 3
#     elif mode == "phase3" or mode == "full":
#         w_qrb1_raw = flat_params[:12].reshape((4, 3))
#         w_qrb2_raw = flat_params[12:24].reshape((4, 3))
#         w_classifier_raw = flat_params[24:].reshape((4, 3))

#         w_qrb1 = qml.numpy.array(w_qrb1_raw, requires_grad=False)
#         w_qrb2 = qml.numpy.array(w_qrb2_raw, requires_grad=False)
#         w_classifier = qml.numpy.array(w_classifier_raw, requires_grad=True)

#         input_layer(x_sequence, wires)
#         qrnn_layer(w_qrb1, wires)
#         interaction_layer(wires)
#         qrnn_layer(w_qrb2, wires)
#         interaction_layer(wires)
#         final_interaction_layer(w_classifier, wires)

# **QNode**

In [ ]:
# dev_test = qml.device("lightning.qubit", wires=4, shots=10000)
# @qml.qnode(dev_test, interface="autograd")
# def stacked_qrnn_test(x_sequence, flat_params):
#     wires = list(range(n_qubits))
#     qrnn_model(x_sequence, flat_params, wires)
#     return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

In [ ]:
n_qubits = 4
# dev = qml.device("default.qubit", wires=n_qubits)
dev_train = qml.device("lightning.qubit", wires=4)
dev_test = qml.device("lightning.qubit", wires=4, shots=1000)

In [ ]:
"""
Simple Qnode with Dense Angle Encoding for the 8 dimensional inputs,
A simple Ring entanglement and parameteried training of input weights
and hidden weights. Returns 2 quantum measurables.
"""

# @qml.qnode(dev, interface="torch")
# def quantum_step(inputs, hidden, weights_in, weights_h):
#     #Encoding
#     dense_angle_embedding(inputs, wires=[0, 1, 2, 3])
#     qml.RY(hidden, wires=3)

#     # Interaction Layer/Entanglement
#     qml.CNOT(wires=[0, 1])
#     qml.CNOT(wires=[1, 2])
#     qml.CNOT(wires=[2, 3])
#     qml.CNOT(wires=[3, 0])

#     # Parametrized trainable unitaries on input wires
#     for i in range(3):
#         qml.RY(weights_in[i], wires=i)
#         qml.RZ(weights_in[i + 3], wires=i)

#     # Parametrized unitaries on hidden wire
#     qml.RY(weights_h[0], wires=3)
#     qml.RZ(weights_h[1], wires=3)

#     return qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

'\nSimple Qnode with Dense Angle Encoding for the 8 dimensional inputs,\nA simple Ring entanglement and parameteried training of input weights\nand hidden weights. Returns 2 quantum measurables.\n'

In [ ]:
"""
Qnode to process a whole sequence at a time in contrast to the initial attempts at
processing a time step at once. It still uses 4 qubit dense angle encoding, but also
preserves the the hidden wire's quantum state throughout the sequence processing,
which helps avoid re-encoding the hidden state at each time-step.
"""

# @qml.qnode(dev, interface="autograd")
# def full_sentence_qnode(sentence_embeds, weights_per_word, classifier_weights):
#     wires = list(range(n_qubits))

#     # Processes a whole sequence (10 words * 8 Dimensional vector embedding)
#     for t in range(len(sentence_embeds)):
#         dense_angle_embedding(sentence_embeds[t], wires)
#         entangle(wires)
#         variational_block(weights_per_word[t], wires)

#     # Final classifier
#     entangle(wires)
#     variational_block(classifier_weights, wires)

#     return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

"\nQnode to process a whole sequence at a time in contrast to the initial attempts at\nprocessing a time step at once. It still uses 4 qubit dense angle encoding, but also\npreserves the the hidden wire's quantum state throughout the sequence processing,\nwhich helps avoid re-encoding the hidden state at each time-step.\n"

In [ ]:
"""
Akin to the previous iteration QNode, this one processes a whole sequence at a time aswell
(10 words - each represented by a 8 dimensional vector). However we have stacked 2 Quantum RNN
layers, each seperated by an "Interaction Layer" where we perform basic ring topology entanglement.
Furthermore there is a final interaction layer, but this layer performs simple RX, RY, RZ rotations
instead of entanglement. This is followed by returning the dual quantum measurables.
"""

@qml.qnode(dev_train, interface="autograd")
def stacked_qrnn_train(x_sequence, flat_params):
    wires = list(range(n_qubits))
    qrnn_model(x_sequence, flat_params, wires)
    return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

@qml.qnode(dev_test, interface="autograd")
def stacked_qrnn_test(x_sequence, flat_params):
    wires = list(range(n_qubits))
    qrnn_model(x_sequence, flat_params, wires)
    return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

In [ ]:
"""
Similar to the previous version, but with an attempt to perform layerwise training in three phases:
a. Phase - 1: Input Layer -> QRNN Layer 1 -> Interaction Layer 1
b. Phase - 2: Input Layer -> QRNN Layer 1 -> Interaction Layer 1 -> QRNN Layer 2 -> Interaction Layer 2
c. Phase - 3: Input Layer -> QRNN Layer 1 -> Interaction Layer 1 -> QRNN Layer 2 -> Interaction Layer 2 -> Interaction Layer 3

Incremently trains and freezes the trainable parameters of previous layers added. This is being performed
in an attempt to mitigate the barren plateau issue being encountered in the previous version of the stacked QRNN.
"""

# @qml.qnode(dev, interface="autograd")
# def stacked_qrnn(x_sequence, flat_params, mode="full"):
#     wires = list(range(n_qubits))
#     qrnn_model(x_sequence, flat_params, wires, mode=mode)
#     return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

'\nSimilar to the previous version, but with an attempt to perform layerwise training in three phases:\na. Phase - 1: Input Layer -> QRNN Layer 1 -> Interaction Layer 1\nb. Phase - 2: Input Layer -> QRNN Layer 1 -> Interaction Layer 1 -> QRNN Layer 2 -> Interaction Layer 2\nc. Phase - 3: Input Layer -> QRNN Layer 1 -> Interaction Layer 1 -> QRNN Layer 2 -> Interaction Layer 2 -> Interaction Layer 3\n\nIncremently trains and freezes the trainable parameters of previous layers added. This is being performed\nin an attempt to mitigate the barren plateau issue being encountered in the previous version of the stacked QRNN.\n'

# **Parameter Initialization**

In [ ]:
# weights_in = np.random.uniform(0, 2 * np.pi, size=6)
# weights_h  = np.random.uniform(0, 2 * np.pi, size=2)
# optimizer = AdamOptimizer(stepsize=0.01)
# epochs = 20

In [ ]:
# weights_per_word = qml.numpy.array(np.random.uniform(0, 2*np.pi, (10, 8)), requires_grad=True)
# classifier_weights = qml.numpy.array(np.random.uniform(0, 2*np.pi, (8,)), requires_grad=True)

# flat_params = qml.numpy.concatenate([
#     weights_per_word.flatten(),
#     classifier_weights.flatten()
# ])

# optimizer = AdamOptimizer(stepsize=0.01)
# epochs = 60

In [ ]:
flat_params = qml.numpy.array(
    np.random.normal(0, 0.01, size=(48,)),
    requires_grad=True
)

optimizer = AdamOptimizer(stepsize=0.005)
epochs = 80

In [ ]:
# Phase - 1
# flat_params = qml.numpy.array(
#     np.random.normal(0, 0.01, size=(12,)),
#     requires_grad=True)

# # Phase - 2
# phase2 = qml.numpy.array(np.random.normal(0, 0.01, size=(12,)), requires_grad=True)
# flat_params = qml.numpy.concatenate([phase1, phase2])

# # Phase - 3
# phase3 = qml.numpy.array(np.random.normal(0, 0.01, size=(12,)), requires_grad=True)
# flat_params = qml.numpy.concatenate([phase1, phase2, phase3])

# optimizer = qml.optimize.AdamOptimizer(stepsize=0.005)
# epochs = 20

# **Directory Creation**

In [ ]:
base_dir = "/content/drive/MyDrive/QML-Research/Analysis"

folders_to_create = [
    "logs/qrnn_outputs",
    "plots/loss_trend",
    "plots/accuracy_trend",
    "plots/test_boxplots",
]

for folder in folders_to_create:
    path = os.path.join(base_dir, folder)
    os.makedirs(path, exist_ok=True)
    print(f"Created (or already exists): {path}")

Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/logs/qrnn_outputs
Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/plots/loss_trend
Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/plots/accuracy_trend
Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/plots/test_boxplots


# **Training**

In [ ]:
train_losses = []
train_accuracies = []

print("Training Loop\n")
# print("Phase 3: Training QRNN Layer 2")
start_time = time.time()

for epoch in range(1, epochs + 1):
    epoch_loss = 0
    correct = 0
    total = 0
    epoch_start = time.time()

    for xb, yb in train_loader:
        x_np = xb.squeeze(0).numpy()
        y_np = yb.item()
        x_qml = qml.numpy.array(x_np, requires_grad=False)

        def cost(flat_params):
          # wpw, clf = reshape_params(flat_params)
          # logits = full_sentence_qnode(x_qml, wpw, clf)
          # logits = stacked_qrnn(x_qml, flat_params, mode = "phase3")
          logits = stacked_qrnn_train(x_qml, flat_params)
          logits = qml.numpy.array(logits)
          probs = softmax(logits)
          return cross_entropy(y_np, probs)

        # weights_in, weights_h = optimizer.step(cost, (weights_in, weights_h))
        # logits = forward(x_np, weights_in, weights_h)
        grad = qml.grad(cost)(flat_params)
        grad_norm = qml.numpy.linalg.norm(grad)
        flat_params = optimizer.step(cost, flat_params)
        # weights_per_word, classifier_weights = reshape_params(flat_params)
        # logits = full_sentence_qnode(x_qml, weights_per_word, classifier_weights)
        # logits = stacked_qrnn(x_qml, flat_params, mode = "phase3")
        logits = stacked_qrnn_train(x_qml, flat_params)
        probs = softmax(logits)
        loss = cross_entropy(y_np, probs)

        pred_label = qml.numpy.argmax(probs)

        if pred_label == y_np:
            correct += 1
        epoch_loss += loss
        total += 1

    avg_loss = epoch_loss / total
    acc = correct / total

    train_losses.append(avg_loss)
    train_accuracies.append(acc)

    epoch_time = time.time() - epoch_start
    print(f"Epoch {epoch:02d} | Loss: {avg_loss:.4f} | Acc: {acc:.4f} | Grad Norm: {grad_norm:.6f} | Time: {epoch_time:.2f}s")

total_time = time.time() - start_time
print(f"\nTraining complete in {total_time:.2f} seconds.")


# print(f"\nPhase 3 complete in {total_time:.2f} seconds.")
# phase1 = qml.numpy.array(flat_params, requires_grad=False)
# phase2 = qml.numpy.array(flat_params[12:], requires_grad=False)
# phase3 = qml.numpy.array(flat_params[24:], requires_grad=False)

# **Phase Weights Save**

In [ ]:
# Saving phase-1 trained parameters
# with open("/content/drive/MyDrive/QML-Research/Model Saves/Phase_Saves/phase1.pkl", "wb") as f:
#     pickle.dump(phase1, f)

# # # Saving phase-2 trained parameters
# with open("/content/drive/MyDrive/QML-Research/Model Saves/Phase_Saves/phase2.pkl", "wb") as f:
#     pickle.dump(phase2, f)

# # Saving phase-3 trained parameters
# with open("/content/drive/MyDrive/QML-Research/Model Saves/Phase_Saves/phase3.pkl", "wb") as f:
#     pickle.dump(phase3, f)

# **Phase Weights Load**

In [ ]:
# # Loading Phase-1 Weights
# with open("/content/drive/MyDrive/QML-Research/Model Saves/Phase_Saves/phase1.pkl", "rb") as f:
#     phase1 = pickle.load(f)
#     phase1 = qml.numpy.array(phase1, requires_grad=False)

# #Loading Phase-2 Weights
# with open("/content/drive/MyDrive/QML-Research/Model Saves/Phase_Saves/phase2.pkl", "rb") as f:
#     phase2 = pickle.load(f)
#     phase2 = qml.numpy.array(phase2, requires_grad=False)

# # #Loading Phase-3 Weights
# with open("/content/drive/MyDrive/QML-Research/Model Saves/Phase_Saves/phase3.pkl", "rb") as f:
#     phase3 = pickle.load(f)
#     phase3 = qml.numpy.array(phase3, requires_grad=False)

# **Testing**

In [ ]:
# def evaluate_test(weights_in, weights_h):
# def evaluate_test(weights_per_word, classifier_weights):
def evaluate_test(flat_params):
    correct = 0
    total = 0
    test_loss = 0

    for xb, yb in test_loader:
        x_np = xb.squeeze(0).numpy()
        y_np = yb.item()
        x_qml = qml.numpy.array(x_np, requires_grad=False)

        # logits = full_sentence_qnode(x_qml, weights_per_word, classifier_weights)
        logits = stacked_qrnn_test(x_qml, flat_params)
        probs = softmax(logits)
        loss = cross_entropy(y_np, probs)

        pred = qml.numpy.argmax(probs)

        if pred == y_np:
            correct += 1
        test_loss += loss
        total += 1

    return test_loss / total, correct / total

# **Logging**

In [ ]:
def log_test_results(model_name, test_acc, test_loss):
    log_path = "/content/drive/MyDrive/QML-Research/Analysis/logs/qrnn_outputs/accuracy_logs.txt"

    with open(log_path, "a") as f:
        f.write(f"[Model: {model_name}]\n")
        f.write(f"Test Accuracy: {test_acc:.4f}\n")
        f.write(f"Test Loss: {test_loss:.4f}\n\n")

    print(f"Logged test results to: {log_path}")

In [ ]:
def save_training_plots(model_name, train_losses, train_accuracies):
    base_path = "/content/drive/MyDrive/QML-Research/Analysis/plots"

    # Loss Plot
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label="Training Loss")
    plt.title(f"{model_name} - Loss Trend")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{base_path}/loss_trend/loss_plot_{model_name}.png")
    plt.close()

    # Accuracy Plot
    plt.figure(figsize=(8, 5))
    plt.plot(train_accuracies, label="Training Accuracy", color='green')
    plt.title(f"{model_name} - Accuracy Trend")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{base_path}/accuracy_trend/acc_plot_{model_name}.png")
    plt.close()

    print(f"Plots saved to: {base_path}/loss_trend/ and /accuracy_trend/")

In [ ]:
# test_loss, test_acc = evaluate_test(weights_in, weights_h)
# test_loss, test_acc = evaluate_test(weights_per_word, classifier_weights)
# flat_params = qml.numpy.concatenate([phase1, phase2, phase3])
test_loss, test_acc = evaluate_test(flat_params)
log_test_results("Stacked_3Layer_v3", test_acc, test_loss)
save_training_plots("Stacked_3Layer_v3", train_losses, train_accuracies)

Logged test results to: /content/drive/MyDrive/QML-Research/Analysis/logs/qrnn_outputs/accuracy_logs.txt
Plots saved to: /content/drive/MyDrive/QML-Research/Analysis/plots/loss_trend/ and /accuracy_trend/


# **Model Saving and Loading**

In [ ]:
with open('/content/drive/MyDrive/QML-Research/Model Saves/3Layer_QRNN_No_Intermediate_Interaction.pkl', 'wb') as f:
    pickle.dump(flat_params, f)

print("Saved Model parameters")

Saved Model parameters


In [ ]:
with open('/content/drive/MyDrive/QML-Research/Models/stacked_qrnn_v6_params.pkl', 'rb') as f:
    flat_params = pickle.load(f)

weights_per_word, classifier_weights = reshape_params(flat_params)
print("Model parameters loaded")

# **Test Inference**

In [ ]:
trial_sentences = [
    "This charger is amazing, super fast and highly reliable.",
    "Worst headphones ever bought, awful sound and terrible build quality.",
    "Very satisfied with camera quality considering the affordable price range.",
    "Product stopped working completely within one week of regular usage.",
    "Excellent screen resolution, great brightness and strong battery backup too.",
    "This is the best light bulb I’ve ever used.",
    "Did not match the description, complete waste of my money.",
    "This chair feels sturdy, comfortable, and worth every single penny.",
    "Absolutely love this phone case, protective, stylish and well-made.",
    "Speaker quality is poor, distorts quickly at medium volume levels.",
    "Delivery arrived quickly with excellent packaging and no visible damages.",
    "Avoid this item entirely, cheap build and constantly stops working.",
    "Best pair of scissors ever purchased, sharp, durable and reliable."
]

def preprocess_for_inference(sent_list):
    encoded_sents = []

    for sentence in sent_list:
        tokens = clean_and_tokenize(sentence)
        if len(tokens) < max_len:
            tokens += ["<PAD>"] * (max_len - len(tokens))
        else:
            tokens = tokens[:max_len]

        vec = sentence_to_vec(tokens, reduced_embeddings, dim=8)
        vec = np.array(vec)
        vec = (vec - X_embed_np.min()) * (np.pi / (X_embed_np.max() - X_embed_np.min()))
        encoded_sents.append(vec)

    return encoded_sents

encoded_inputs = preprocess_for_inference(trial_sentences)

for sent, input_vec in zip(trial_sentences, encoded_inputs):
    x_qml = qml.numpy.array(input_vec, requires_grad=False)
    logits = stacked_qrnn_test(x_qml, flat_params)
    probs = softmax(logits)
    pred = qml.numpy.argmax(probs)

    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Sentence: \"{sent}\"\nPredicted Sentiment: {sentiment} ({pred})\n")

Sentence: "This charger is amazing, super fast and highly reliable."
Predicted Sentiment: Negative (0)

Sentence: "Worst headphones ever bought, awful sound and terrible build quality."
Predicted Sentiment: Positive (1)

Sentence: "Very satisfied with camera quality considering the affordable price range."
Predicted Sentiment: Negative (0)

Sentence: "Product stopped working completely within one week of regular usage."
Predicted Sentiment: Positive (1)

Sentence: "Excellent screen resolution, great brightness and strong battery backup too."
Predicted Sentiment: Negative (0)

Sentence: "This is the best light bulb I’ve ever used."
Predicted Sentiment: Negative (0)

Sentence: "Did not match the description, complete waste of my money."
Predicted Sentiment: Negative (0)

Sentence: "This chair feels sturdy, comfortable, and worth every single penny."
Predicted Sentiment: Negative (0)

Sentence: "Absolutely love this phone case, protective, stylish and well-made."
Predicted Sentiment: Nega